<a href="https://colab.research.google.com/github/Macdaline100/Public-Data/blob/master/Macdaline_Raisibe_Mathye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Science Academy Classification Hackathon 

### Let's do some importing 

In [1]:
#importing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import nltk
import re
import string
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk import word_tokenize, pos_tag, pos_tag_sents
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import resample
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


### Let's do some data loading

In [2]:
train = pd.read_csv("train_set.csv")
test = pd.read_csv("test_set.csv")


In [3]:
test = pd.read_csv("test_set.csv")

### Exploring

In [4]:
train['lang_id'].value_counts()

xho    1550
tso    1541
zul    1536
tsn    1525
ven    1523
afr    1499
eng    1496
ssw    1488
nso    1488
nbl    1482
sot    1474
Name: lang_id, dtype: int64

In [5]:
train.isnull().sum()

lang_id    0
text       0
dtype: int64

In [6]:
train.head(2)

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...


In [7]:
%%capture
!pip install --upgrade -q comet_ml contractions emoji unidecode langdetect

In [8]:
!!pip install nlppreprocess
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import contractions

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [9]:

def clean_tweets(message):
    """
    Cleaning all tweets by removing contractions, url-links, punctuation, digits,
    stopwords and Lemmatizing all the words.

    Returns
      A clean tweet as string
    """

    # change all words into lower case
    message = message.lower()

    #removing contractions
    message = contractions.fix(message)

    # replace all url-links with url-web
    url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
    web = 'url-web'
    message = re.sub(url, web, message)

    # removing all punctuation and digits
    message = re.sub(r'[-]',' ',message)
    message = re.sub(r'[^\w\s]','',message)
    message = re.sub('[0-9]+', '', message)
    
    # removing stopwords
    nltk_stopword = nltk.corpus.stopwords.words('english')
    message = ' '.join([item for item in message.split() if item not in nltk_stopword])

    # lemmatizing all words
    message = message.lower()
    lemmatizer = WordNetLemmatizer()
    message = [lemmatizer.lemmatize(token) for token in message.split(" ")]
    message = [lemmatizer.lemmatize(token, "v") for token in message]
    message = " ".join(message)

    return message
train['text_clean']=train['text'].apply(clean_tweets)

In [10]:
def TweetCleaner(tweet):
    
    """
    This function uses regular expressions to remove url's, mentions, hashtags, 
    punctuation, numbers and any extra white space from tweets after converting 
    everything to lowercase letters.

    Input:
    tweet: original tweet
           datatype: 'str'

    Output:
    tweet: modified tweet
           datatype: 'str'
    """
    # Convert everything to lowercase
    tweet = tweet.lower() 
    
    # Remove mentions   
    tweet = re.sub('@[\w]*','',tweet)  
    
    # Remove url's
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    
    # Remove hashtags
    tweet = re.sub(r'#\w*', '', tweet)    
    
    # Remove numbers
    tweet = re.sub(r'\d+', '', tweet)  
    
    # Remove punctuation
    tweet = re.sub(r"[,.;':@#?!\&/$]+\ *", ' ', tweet)
    
    # Remove that funny diamond
    tweet = re.sub(r"U+FFFD ", ' ', tweet)
    
    # Remove extra whitespace
    tweet = re.sub(r'\s\s+', ' ', tweet)
    
    # Remove space in front of tweet
    tweet = tweet.lstrip(' ')                        
    
    return tweet

# Clean the tweets in the message column
train['text_token'] = train['text'].apply(TweetCleaner)

In [11]:
#tokenizing the tweets
>>> from nltk.tokenize import WordPunctTokenizer

tokeniser =WordPunctTokenizer() # TreebankWordTokenizer()

train['tokenized'] = train['text'].apply(tokeniser.tokenize)

In [12]:
train.head()

,lang_id,text,text_clean,text_token,tokenized
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,umgaqo siseko wenza amalungiselelo kumaziko ax...,umgaqo-siseko wenza amalungiselelo kumaziko ax...,"[umgaqo, -, siseko, wenza, amalungiselelo, kum..."
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...,dha iya kuba nobulumko bokubeka umsebenzi naph...,i-dha iya kuba nobulumko bokubeka umsebenzi na...,"[i, -, dha, iya, kuba, nobulumko, bokubeka, um..."
2,eng,the province of kwazulu-natal department of tr...,province kwazulu natal department transport in...,the province of kwazulu-natal department of tr...,"[the, province, of, kwazulu, -, natal, departm..."
3,nso,o netefatša gore o ba file dilo ka moka tše le...,netefatša gore ba file dilo ka moka tše le dum...,o netefatša gore o ba file dilo ka moka tše le...,"[o, netefatša, gore, o, ba, file, dilo, ka, mo..."
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...,khomishini ya ndinganyiso ya mbeu yo ewa maana...,khomishini ya ndinganyiso ya mbeu yo ewa maana...,"[khomishini, ya, ndinganyiso, ya, mbeu, yo, ew..."


### Training

In [46]:
X = train['text']
y = train['lang_id'] 

In [49]:
#Splitting features and target variables

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,test_size = 0.05, random_state = 10)

In [17]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer( ngram_range=(1, 3), analyzer='char', max_df = 0.70, tokenizer=TreebankWordTokenizer().tokenize) #tokenizer=word_tokenize) # tokenizer=TreebankWordTokenizer().tokenize) #=WordPunctTokenizer() 

In [18]:
#X_train=TfidfVectorizer( ngram_range=(1, 3), analyzer='word', max_df = 0.70, tokenizer=WordPunctTokenizer().tokenize).fit_transform( X_train.values)
#X_test=TfidfVectorizer(ngram_range=(1, 3), analyzer='word', max_df = 0.70, tokenizer=WordPunctTokenizer().tokenize).fit_transform( X_test.values)

In [109]:
paramgrid = { 'alpha': [0.01, 0.1, 0.2,0.3,1,400,0.6,60, 0.8,300,40]}
multi = Pipeline([('tfidf', CountVectorizer(lowercase = True, ngram_range=(1, 3), analyzer='word',max_df = 0.7)),
                 ('clf', GridSearchCV(MultinomialNB(),
                                     param_grid=paramgrid,
                                     cv=200,
                                     n_jobs= -1))])

multi.fit(X_train, y_train)
y_pred = multi.predict(X_test)
print('f1_score %s' % metrics.f1_score(y_test,y_pred,average='weighted')) 
print(classification_report(y_test, y_pred),2)
multi_nb = classification_report(y_test, y_pred, output_dict= True)

f1_score 0.9993939384898343
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       900
         eng       1.00      1.00      1.00       900
         nbl       1.00      1.00      1.00       900
         nso       1.00      1.00      1.00       900
         sot       1.00      1.00      1.00       900
         ssw       1.00      1.00      1.00       900
         tsn       1.00      1.00      1.00       900
         tso       1.00      1.00      1.00       900
         ven       1.00      1.00      1.00       900
         xho       1.00      1.00      1.00       900
         zul       1.00      1.00      1.00       900

    accuracy                           1.00      9900
   macro avg       1.00      1.00      1.00      9900
weighted avg       1.00      1.00      1.00      9900
 2


In [29]:
from sklearn.ensemble import RandomForestClassifier
rfc  = RandomForestClassifier( )
rfc  = Pipeline([('tfidf', tfidf), ('clf', rfc)]) 
rfc.fit(X_train, y_train) 
rfc_pred = rfc.predict(X_test)
print(metrics.accuracy_score(y_test ,rfc_pred))
print(f1_score(y_test, rfc_pred, average="weighted"))
print(classification_report(y_test, rfc_pred)) 

0.982836495031617
0.9829015108726087
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       300
         eng       0.99      1.00      0.99       299
         nbl       0.99      0.92      0.95       296
         nso       1.00      0.99      1.00       298
         sot       0.99      1.00      1.00       295
         ssw       0.99      0.96      0.98       298
         tsn       1.00      1.00      1.00       305
         tso       1.00      1.00      1.00       308
         ven       1.00      1.00      1.00       305
         xho       0.95      0.97      0.96       310
         zul       0.91      0.97      0.94       307

    accuracy                           0.98      3321
   macro avg       0.98      0.98      0.98      3321
weighted avg       0.98      0.98      0.98      3321



In [30]:
lr = LogisticRegression()
clf_lr = Pipeline([('tfidf', tfidf), ('clf', lr)]) 
clf_lr.fit(X_train, y_train) 
y_pred_lr= clf_lr.predict(X_test)
# printing accuracy and score
print('accuracy %s' % accuracy_score(y_pred_lr, y_test)) 
print('f1_score %s' % metrics.f1_score(y_test,y_pred_lr,average='weighted')) 
print(classification_report(y_test, y_pred_lr)) 

accuracy 0.992773261065944
f1_score 0.9927694225220682
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       300
         eng       0.99      1.00      0.99       299
         nbl       0.99      0.97      0.98       296
         nso       1.00      0.99      1.00       298
         sot       1.00      1.00      1.00       295
         ssw       1.00      0.99      0.99       298
         tsn       0.99      1.00      1.00       305
         tso       1.00      1.00      1.00       308
         ven       1.00      1.00      1.00       305
         xho       0.98      0.99      0.98       310
         zul       0.98      0.98      0.98       307

    accuracy                           0.99      3321
   macro avg       0.99      0.99      0.99      3321
weighted avg       0.99      0.99      0.99      3321



In [ ]:
solvers = [ 'liblinear']
penalty = ['l2']
c_values = [100, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)


#c = { 'alpha': [0.01, 0.1, 0.2,0.3,1,300,40]}
lh = Pipeline([('tfidf', CountVectorizer(lowercase = True, ngram_range=(1, 3), analyzer='word', max_df = 0.7)),
                 ('clf', GridSearchCV(LogisticRegression(),
                                     param_grid=grid,
                                     cv=100,
                                     n_jobs= -1))])

lh.fit(X_train, y_train)
y_pred = multi.predict(X_test)
print('f1_score %s' % metrics.f1_score(y_test,y_pred,average='weighted')) 
print(classification_report(y_test, y_pred),2)
multi_nb = classification_report(y_test, y_pred, output_dict= True)

In [31]:
from sklearn.svm import SVC
svc= SVC(kernel='rbf')#{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}
clf_svc= Pipeline([('tfidf', tfidf), ('clf', svc)])
clf_svc.fit(X_train, y_train)
y_pred_svc = clf_svc.predict(X_test)
#printing accuracy and score
print('accuracy %s' % accuracy_score(y_pred_svc, y_test)) 
print('f1_score %s' % metrics.f1_score(y_test,y_pred_svc,average='weighted')) 
print(classification_report(y_test, y_pred_svc)) 

accuracy 0.9939777175549533
f1_score 0.9939769246358623
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       300
         eng       1.00      1.00      1.00       299
         nbl       0.99      0.97      0.98       296
         nso       1.00      1.00      1.00       298
         sot       1.00      1.00      1.00       295
         ssw       1.00      0.99      0.99       298
         tsn       1.00      1.00      1.00       305
         tso       1.00      1.00      1.00       308
         ven       1.00      1.00      1.00       305
         xho       0.98      1.00      0.99       310
         zul       0.97      0.98      0.98       307

    accuracy                           0.99      3321
   macro avg       0.99      0.99      0.99      3321
weighted avg       0.99      0.99      0.99      3321



In [32]:
nb = MultinomialNB()
clf_nb= Pipeline([('tfidf', tfidf), ('clf', nb)])
clf_nb.fit(X_train, y_train)
y_pred_nb = clf_nb.predict(X_test)
#printing accuracy and score
print('accuracy %s' % accuracy_score(y_pred_nb, y_test)) 
print('f1_score %s' % metrics.f1_score(y_test,y_pred_nb,average='weighted')) 
print(classification_report(y_test, y_pred_nb)) 

accuracy 0.9978922011442337
f1_score 0.9978921751804353
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       300
         eng       0.99      1.00      1.00       299
         nbl       1.00      0.99      1.00       296
         nso       1.00      1.00      1.00       298
         sot       0.99      1.00      1.00       295
         ssw       1.00      0.99      1.00       298
         tsn       1.00      1.00      1.00       305
         tso       1.00      1.00      1.00       308
         ven       1.00      1.00      1.00       305
         xho       1.00      1.00      1.00       310
         zul       1.00      1.00      1.00       307

    accuracy                           1.00      3321
   macro avg       1.00      1.00      1.00      3321
weighted avg       1.00      1.00      1.00      3321



### Hyperparameter Tuning

In [33]:
paramgrid = { 'alpha': [0.01, 0.1, 0.2,0.3,1,400,0.6,0.8,300,40]}
multi = Pipeline([('tfidf', CountVectorizer(lowercase = True, ngram_range=(1, 3), analyzer='word', stop_words = 'english',max_df = 0.7)),
                 ('clf', GridSearchCV(MultinomialNB(),
                                     param_grid=paramgrid,
                                     cv=100,
                                     n_jobs= -1))])

multi.fit(X_train, y_train)
y_pred = multi.predict(X_test)
print('f1_score %s' % metrics.f1_score(y_test,y_pred,average='weighted')) 
print(classification_report(y_test, y_pred),2)
multi_nb = classification_report(y_test, y_pred, output_dict= True)

f1_score 0.9987950066944952
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       300
         eng       1.00      1.00      1.00       299
         nbl       1.00      0.99      1.00       296
         nso       1.00      1.00      1.00       298
         sot       1.00      1.00      1.00       295
         ssw       1.00      1.00      1.00       298
         tsn       1.00      1.00      1.00       305
         tso       1.00      1.00      1.00       308
         ven       1.00      1.00      1.00       305
         xho       1.00      1.00      1.00       310
         zul       1.00      1.00      1.00       307

    accuracy                           1.00      3321
   macro avg       1.00      1.00      1.00      3321
weighted avg       1.00      1.00      1.00      3321
 2


In [59]:
test_new = cv.transform(test['text'])

In [72]:
predictions = multi.predict(test_new)
submission = pd.DataFrame({'index':test.index,'lang_id':predictions})
submission = submission.set_index('index')
submission.tail()

,lang_id
index,
5677,eng
5678,nso
5679,sot
5680,sot
5681,nbl


In [ ]:
submission.to_csv('JUL2021_Raisibe_Macdaline_Mathye.csv',index=True)
submission

In [158]:
# Make Submission
My_submission = pd.DataFrame(test['index'])
My_submission['lang_id'] = multi.predict(test['text'])
My_submission.to_csv('JUL2021_Raisibe_Macdaline_Mathye.csv',index=False)